In [1]:
# main program:
# importing libs:
import cv2
import numpy as np
import torch
from time import time
from ultralytics import YOLO

# loading a yolo model:
# using yolov8-nano model:
# class for distance calculation:
class distanceMeasurement:
    def __init__(self):
        # initializing yolo model:
        self.model = YOLO('./yolov10n.pt')

        # Known parameters - these would be calibrated for your specific setup
        # Focal length (pixels) = (image width in pixels) * (real width / apparent width)
        self.focal_length = 1000  # Example value, needs to be calibrated
        
        # Known width of objects in inches/cm
        self.KNOWN_WIDTH = {
            'person': 60,  # average width in cm
            'car': 180,    # average width in cm
            'bottle': 8,   # average width in cm
            'laptop': 35   # average width in cm
        }
        
        # Default width for objects not in our dictionary
        self.DEFAULT_WIDTH = 30  # cm

    # function for calculating distance:
    def calculate_distance(self, bbox_width, object_class):
        """
        Calculate distance based on known object width and apparent width in image
        Using the formula: distance = (known width × focal length) / apparent width
        """
        real_width = self.KNOWN_WIDTH.get(object_class, self.DEFAULT_WIDTH)
        distance = (real_width * self.focal_length) / bbox_width
        return distance
    
    # processing result frame:
    def process_frame(self, frame):
        """Process a single frame, detect objects and estimate distances"""
        # Make a copy of the frame to draw on
        result_frame = frame.copy()
        
        # Run object detection with YOLOv8
        results = self.model(frame)
        
        # Process each detection from the first (and only) image result
        for r in results:
            boxes = r.boxes
            
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                
                # Get confidence score
                conf = box.conf[0].item()
                
                # Get class name
                class_id = int(box.cls[0].item())
                class_name = self.model.names[class_id]
                
                if conf < 0.5:  # Filter low confidence detections
                    continue
                
                # Calculate bbox width
                bbox_width = x2 - x1
                
                # Calculate distance
                distance = self.calculate_distance(bbox_width, class_name)
                
                # Draw bounding box
                cv2.rectangle(result_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                
                # Draw label with class name and distance
                label = f"{class_name}: {distance:.2f} cm"
                cv2.putText(result_frame, label, (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        return result_frame
    
    # calibrate focal length:
    def calibrate_focal_length(self, known_distance, known_width, apparent_width):
        """
        Calibrate the focal length using an object of known size at a known distance
        focal_length = (apparent width * known distance) / known width
        """
        self.focal_length = (apparent_width * known_distance) / known_width
        print(f"Focal length calibrated to: {self.focal_length}")
        return self.focal_length
    
    # process the final output
    def process_video(self, video_source=1):
        """Process live webcam feed with stabilized settings"""
        # 1. Use DirectShow for Windows stability
        cap = cv2.VideoCapture(video_source, cv2.CAP_DSHOW)
        
        # 2. FORCE MJPG format to fix the 'rainbow static' / buffer error
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
        
        # 3. Explicitly set resolution BEFORE starting the loop
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        if not cap.isOpened():
            print(f"Error: Could not open video source {video_source}.")
            return

        print("Press 'q' to quit...")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Warning: Empty frame received.")
                break
            
            # Start timing for FPS
            start_time = time()
            
            # Process the frame for detections and distance
            result_frame = self.process_frame(frame)
            
            end_time = time()
            
            # Calculate and display FPS
            processing_time = end_time - start_time
            fps_text = f"FPS: {1/processing_time:.2f}" if processing_time > 0 else "FPS: 0"
            cv2.putText(result_frame, fps_text, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            # Display the live result
            cv2.imshow("Live Distance Measurement", result_frame)
            
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

# initializing the program:
if __name__ == "__main__":
    distance_detector = distanceMeasurement()
    
    # Standard webcam is usually 0. 
    # Try 0 if 1 or 2 does not work.
    distance_detector.process_video(video_source=1)

# initializing the program:
if __name__ == "__main__":
    distance_detector = distanceMeasurement()
    
    # Optional: Calibrate with a known object at a known distance
    # Example: A person (60cm wide) at 300cm distance appears 200 pixels wide
    # distance_detector.calibrate_focal_length(known_distance=300, known_width=60, apparent_width=200)
    
    # Process video
    video_source = 1  # Use 0 for webcam or a path to a video file
    distance_detector.process_video(video_source)


Press 'q' to quit...

0: 480x640 1 person, 436.9ms
Speed: 37.4ms preprocess, 436.9ms inference, 8.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 307.4ms
Speed: 7.9ms preprocess, 307.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 464.3ms
Speed: 3.4ms preprocess, 464.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 416.9ms
Speed: 6.0ms preprocess, 416.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 486.4ms
Speed: 4.3ms preprocess, 486.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 537.2ms
Speed: 8.8ms preprocess, 537.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 473.7ms
Speed: 5.6ms preprocess, 473.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 230.3ms
Speed: 4.0ms preprocess, 230.3ms inference, 0